# FastAPI

- Основы
    - Path operation
    - Request\Response
    - Models
    - Pydantic
    - Dependencies
    - Deploy
- Работа с БД
    - SQLAlchemy
        - Подключение
        - CRUD
        - sync\async
    - Authentication schemas
        - Basic
        - Bearer
        - OAuth2
        - JWT

## Основы

**Цели занятия**:
- Запустить простое приложение
- Провалидировать модель запроса и ответа с помощью Pydantic
- Добавить простую аутентификаицю
- Собрать docker-контейнер с приложением

Documentation: https://fastapi.tiangolo.com/

Tutorial: https://fastapi.tiangolo.com/tutorial/

**Чем хорош FastAPI?**

- Современный веб-фреймворк для быстрой разработки
- Достаточно быстрый
- Много фич из коробки
- autodocs

### Установка

In [ ]:
pip install "fastapi[all]"
pip install "uvicorn[standard]"

### Hello world

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}

### Запуск

In [ ]:
uvicorn main:app --reload

либо

In [ ]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run("app.main:app", host="0.0.0.0", port=8000, reload=True, log_level='error')

либо с помощью gunicorn

### Request parameters

1. Path parameters
2. Query string
3. Request body
    - Form
    - Files
    - JSON
4. Headers

#### Path parameters

Позволяет извлекать параметры из пути, по которому произошло обращение на сервер, используя плейсхолдеры с именами параметров, передаваемыми в качестве аргументов в функции-обработчики запросов

In [ ]:
@app.get("/items/{item_id}")
async def read_item(item_id):
    return {"item_id": item_id}

In [20]:
! wget -q -S -O - http://172.27.10.31:8001/items/ret83g7f67e

  HTTP/1.1 422 Unprocessable Entity
  date: Tue, 10 Oct 2023 16:09:27 GMT
  server: uvicorn
  content-length: 214
  content-type: application/json


In [ ]:
@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}

In [19]:
! wget -q -S -O - http://172.27.10.31:8001/items/678

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:09:17 GMT
  server: uvicorn
  content-length: 15
  content-type: application/json
{"item_id":678}

In [18]:
! wget -q -S -O - http://172.27.10.31:8001/items/ce6e56

  HTTP/1.1 422 Unprocessable Entity
  date: Tue, 10 Oct 2023 16:09:10 GMT
  server: uvicorn
  content-length: 209
  content-type: application/json


В общем случае, необходимо помнить, что:
- Метод и путь, привязываемые к обработчику запроса, задаются с помощью декоратора @app.{method}
- Допустимо называть функции-обработчики одинаковым именем
- Допустимо задавать у нескольких функций-обарботчиков одинаковые параметры (метод, путь)
- При этом работать будет только самый верхний обработчик

In [ ]:
@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}


@app.get("/items/{request_id}")
async def read_item(request_id):
    '''
    This request handler will never be reached
    :param request_id:
    :return:
    '''
    return {"request_id": request_id}

В случае, если по каким-либо причинам вам необходим отдельный обработчик для запроса, путь которого подходит под уже имеющийся, но при этом значения в плейсхолдерах будут конкретными, например:
- /users/{user_id}
- /users/current

Разместите более конкретный обработчик над общим

In [ ]:
@app.get("/requests/current")
async def read_item():
    return {"request": "current item"}


@app.get("/requests/{request_id}")
async def read_item(request_id):
    return {"request": request_id}

In [22]:
! wget -q -S -O - http://172.27.10.31:8001/requests/alpha-14

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:18:34 GMT
  server: uvicorn
  content-length: 22
  content-type: application/json
{"request":"alpha-14"}

In [23]:
! wget -q -S -O - http://172.27.10.31:8001/requests/current

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:18:44 GMT
  server: uvicorn
  content-length: 26
  content-type: application/json
{"request":"current item"}

Можно ограничить вводимые в path parameter значение с помощью enum

In [ ]:
class RequestTypes(str, Enum):
    service = 'service'
    new_feature = 'new_feature'


@app.get("/requests/type/{request_type}")
async def read_item(request_type: RequestTypes):
    return {"request_typpe": request_type.value}

In [24]:
! wget -q -S -O - http://172.27.10.31:8001/requests/type/service

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:22:58 GMT
  server: uvicorn
  content-length: 27
  content-type: application/json
{"request_typpe":"service"}

In [25]:
! wget -q -S -O - http://172.27.10.31:8001/requests/type/abrakadabra

  HTTP/1.1 422 Unprocessable Entity
  date: Tue, 10 Oct 2023 16:23:09 GMT
  server: uvicorn
  content-length: 179
  content-type: application/json


#### Query string

Query string - пары ключ=значение, следующие сразу после пути.

```/api/requests?param1=val1&param2=val2```

In [ ]:
@app.get("/query_params")
async def read_item(page: int = 0, skip: int = 0):
    return {
        "page": page,
        "skip": skip
    }

In [ ]:
! wget -q -S -O - http://172.27.10.31:8001/query_params

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:28:49 GMT
  server: uvicorn
  content-length: 19
  content-type: application/json
{"page":0,"skip":0}

In [27]:
! wget -q -S -O - http://172.27.10.31:8001/query_params?page=10

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:28:59 GMT
  server: uvicorn
  content-length: 20
  content-type: application/json
{"page":10,"skip":0}

In [2]:
! wget -q -S -O - "http://172.27.10.31:8001/query_params?page=10&skip=1"

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:39:47 GMT
  server: uvicorn
  content-length: 20
  content-type: application/json
{"page":10,"skip":1}

Query parameters могут быть обязательными и необязательными. В предыдущих примерах возможно было не передавать один или оба параметра. В этом случае они будут инициализированы значениями по-умолчанию.

В случае, если требуется обязательное присутствие параметра - значение по-умолчанию не задается.

In [ ]:
@app.get("/query_params")
async def read_item(req: str, page: int = 0, skip: int = 0):
    return {
        "page": page,
        "skip": skip,
        "req": req
    }

In [3]:
! wget -q -S -O - "http://172.27.10.31:8001/query_params?page=10&skip=1"

  HTTP/1.1 422 Unprocessable Entity
  date: Tue, 10 Oct 2023 16:45:22 GMT
  server: uvicorn
  content-length: 139
  content-type: application/json


In [4]:
! wget -q -S -O - "http://172.27.10.31:8001/query_params?req=test"

  HTTP/1.1 200 OK
  date: Tue, 10 Oct 2023 16:45:36 GMT
  server: uvicorn
  content-length: 32
  content-type: application/json
{"page":0,"skip":0,"req":"test"}

#### Query & Path validation

Некоторые параметры требуется заранее ограничивать по диапазону принимаемых значений, для этого в качестве типа параметра используется ```Query``` в сочетании с ```Annotated```.

In [ ]:
from typing import Annotated

from fastapi import Query, Path

In [ ]:
@app.get("/items/")
async def read_items(q: Annotated[str | None, Query(max_length=50)] = None):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

In [ ]:
@app.get("/query_params_typed")
async def read_item(req: Annotated[str, Query(min_length=5, max_length=15)]):
    return {
        "req": req
    }

In [6]:
! wget -q -S -O - "http://172.27.10.31:8001/query_params_typed?req=aaa"

  HTTP/1.1 422 Unprocessable Entity
  date: Wed, 11 Oct 2023 07:55:32 GMT
  server: uvicorn
  content-length: 207
  content-type: application/json


In [7]:
! wget -q -S -O - "http://172.27.10.31:8001/query_params_typed?req=aaaaa"

  HTTP/1.1 200 OK
  date: Wed, 11 Oct 2023 07:55:41 GMT
  server: uvicorn
  content-length: 15
  content-type: application/json
{"req":"aaaaa"}

In [ ]:
@app.get("/items_validated/{request_id}")
async def read_item(request_id: Annotated[int, Path(ge=10, lt=15)]):
    return {"request_id": request_id}

In [1]:
! wget -q -S -O - "http://172.27.10.31:8001/items_validated/10"

  HTTP/1.1 200 OK
  date: Wed, 11 Oct 2023 08:03:54 GMT
  server: uvicorn
  content-length: 17
  content-type: application/json
{"request_id":10}

In [2]:
! wget -q -S -O - "http://172.27.10.31:8001/items_validated/100"

  HTTP/1.1 422 Unprocessable Entity
  date: Wed, 11 Oct 2023 08:04:01 GMT
  server: uvicorn
  content-length: 180
  content-type: application/json


#### Request body

- Form data
- Files
- JSON

Для обращения к объекту "Запрос", содежращему в себе все данные запроса, можно добавить аргумент типа Request

In [ ]:
@app.post("/")
async def read_item(request: Request):
    return {
        "headers": request.headers,
        "cooke": request.cookies,
        "body": await request.body()
    }

In [6]:
! wget --post-data="user=evgeniy&password=qwerty" -q -S -O - "http://172.27.10.31:8001/"

  HTTP/1.1 200 OK
  date: Wed, 11 Oct 2023 11:33:13 GMT
  server: uvicorn
  content-length: 259
  content-type: application/json
{"headers":{"host":"172.27.10.31:8001","user-agent":"Wget/1.21.2","accept":"*/*","accept-encoding":"identity","connection":"Keep-Alive","content-type":"application/x-www-form-urlencoded","content-length":"28"},"cooke":{},"body":"user=evgeniy&password=qwerty"}

In [7]:
! wget --post-data="{\"user\":\"otus\",\"password\":\"otus\"}" -q -S -O - "http://172.27.10.31:8001/"

  HTTP/1.1 200 OK
  date: Wed, 11 Oct 2023 11:33:59 GMT
  server: uvicorn
  content-length: 272
  content-type: application/json
{"headers":{"host":"172.27.10.31:8001","user-agent":"Wget/1.21.2","accept":"*/*","accept-encoding":"identity","connection":"Keep-Alive","content-type":"application/x-www-form-urlencoded","content-length":"33"},"cooke":{},"body":"{\"user\":\"otus\",\"password\":\"otus\"}"}

Данный путь, хоть и является достаточно интуитивным, но весьма топорный. Можно параметризовать обработчик запроса, также как и в случае с Path и Query.

In [ ]:
from fastapi import Form

@app.post("/form")
async def read_item(username: Annotated[str, Form()], password: Annotated[str, Form()]):
    return {
        "username": username,
        "password": password
    }

In [8]:
! wget --post-data="username=evgeniy&password=qwerty" -q -S -O - "http://172.27.10.31:8001/form"

  HTTP/1.1 200 OK
  date: Wed, 11 Oct 2023 11:43:28 GMT
  server: uvicorn
  content-length: 42
  content-type: application/json
{"username":"evgeniy","password":"qwerty"}

In [ ]:
from fastapi import File

@app.post("/files/")
async def create_file(file: Annotated[bytes, File()]):
    return {"file_size": len(file)}

In [10]:
! ls -la | tail -n 3

-rw-r--r--  1 jovyan users       72 Mar  3  2023 Untitled.ipynb
-rw-r--r--  1 jovyan users   552944 Jul 11 17:58 Web.ipynb
drwxr-sr-x  1 jovyan users       72 Sep  5 18:13 С-ext | FFI


In [16]:
! pip install httpie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.8 MB/s eta 0:00:00


In [19]:
! http -f POST http://172.27.10.31:8001/files/ file@Web.ipynb

HTTP/1.1 200 OK
content-length: 20
content-type: application/json
date: Wed, 11 Oct 2023 11:49:29 GMT
server: uvicorn

{
    "file_size": 552944
}




### Pydantic

Так как для построения REST API скорее всего будет использоваться JSON-формат отправки данных - парсить запрос можно с помощью

```json.loads(await request.body().decode('utf-8')```
              
Но, этот метод также достаточно топорный.
            
FastAPI предлагает возможность автоматически десериализовать JSON из запроса в экземляр некоторого класса. За десериализацию, валидацю, выдачу ошибок, также, как и в случае с Path, Query, отвечает Pydantic.

In [ ]:
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


@app.post("/items/")
async def create_item(
    item: Item | None = None,
):
    return {"item": item}

In [22]:
! http POST http://172.27.10.31:8001/items/ \
    name="test item" \
    description="test description" \
    price:=29 \
    tax:=0.1

HTTP/1.1 200 OK
content-length: 85
content-type: application/json
date: Wed, 11 Oct 2023 12:00:39 GMT
server: uvicorn

{
    "item": {
        "description": "test description",
        "name": "test item",
        "price": 29.0,
        "tax": 0.1
    }
}




In [25]:
! http POST http://172.27.10.31:8001/items/ \
    name="test item" \
    description="test description" \
    price:=29 \
    tax:=true

HTTP/1.1 200 OK
content-length: 85
content-type: application/json
date: Wed, 11 Oct 2023 12:01:20 GMT
server: uvicorn

{
    "item": {
        "description": "test description",
        "name": "test item",
        "price": 29.0,
        "tax": 1.0
    }
}




In [29]:
! http POST http://172.27.10.31:8001/items/ \
    name="test item" \
    description="test description" \
    price:=29 \
    pax:=0

HTTP/1.1 200 OK
content-length: 86
content-type: application/json
date: Wed, 11 Oct 2023 12:01:55 GMT
server: uvicorn

{
    "item": {
        "description": "test description",
        "name": "test item",
        "price": 29.0,
        "tax": null
    }
}




In [33]:
! http POST http://172.27.10.31:8001/items/ \
    field="123"

HTTP/1.1 422 Unprocessable Entity
content-length: 289
content-type: application/json
date: Wed, 11 Oct 2023 12:10:32 GMT
server: uvicorn

{
    "detail": [
        {
            "input": {
                "field": "123"
            },
            "loc": [
                "body",
                "name"
            ],
            "msg": "Field required",
            "type": "missing",
            "url": "https://errors.pydantic.dev/2.4/v/missing"
        },
        {
            "input": {
                "field": "123"
            },
            "loc": [
                "body",
                "price"
            ],
            "msg": "Field required",
            "type": "missing",
            "url": "https://errors.pydantic.dev/2.4/v/missing"
        }
    ]
}




Можно создавать вложенные модели. FastAPI распарсит их из объекта верхнего уровня, при этом часть параметров можно задать в качестве отдельных аргументов с помощью ```Body()```

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Body
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


class User(BaseModel):
    username: str
    full_name: str | None = None


@app.put("/items/{item_id}")
async def update_item(
    item_id: int, item: Item, user: User, importance: Annotated[int, Body()]
):
    results = {"item_id": item_id, "item": item, "user": user, "importance": importance}
    return results

In [ ]:
Request body:

{
    "item": {
        "name": "Foo",
        "description": "The pretender",
        "price": 42.0,
        "tax": 3.2
    },
    "user": {
        "username": "dave",
        "full_name": "Dave Grohl"
    },
    "importance": 5
}

In [35]:
! http PUT http://172.27.10.31:8001/items/1 \
    item[name]="foo" \
    item[description]="The pretender" \
    item[description]="The pretender" \
    item[price]:=42.0 \
    item[price]:=3.2 \
    user[username]="dave" \
    user[full_name]="Dave Grohl" \
    importance:=5

HTTP/1.1 200 OK
content-length: 155
content-type: application/json
date: Wed, 11 Oct 2023 12:13:56 GMT
server: uvicorn

{
    "importance": 5,
    "item": {
        "description": "The pretender",
        "name": "foo",
        "price": 3.2,
        "tax": null
    },
    "item_id": 1,
    "user": {
        "full_name": "Dave Grohl",
        "username": "dave"
    }
}




#### Response

Задать тип ответа можно как с помощью type-hint возвращаемого значения, так и с помощью keyword-аргумента ```response_model``` в декораторе.

В случае, если указаны оба значения - ```response_model``` будет в приоритете.

In [ ]:
@app.get("/items/", response_model=list[Item])
async def read_items() -> Any:
    return [
        {"name": "Portal Gun", "price": 42.0},
        {"name": "Plumbus", "price": 32.0},
    ]

In [36]:
! http GET http://172.27.10.31:8001/items/

HTTP/1.1 200 OK
content-length: 128
content-type: application/json
date: Wed, 11 Oct 2023 12:17:18 GMT
server: uvicorn

[
    {
        "description": null,
        "name": "Portal Gun",
        "price": 42.0,
        "tax": null
    },
    {
        "description": null,
        "name": "Plumbus",
        "price": 32.0,
        "tax": null
    }
]




#### Other responses

FastAPI поддерживает еще несколько типов ответов:
- JSONResonse
- FileResponse
- RedirectResponse
- ...

https://fastapi.tiangolo.com/advanced/custom-response/?h=jsonres#available-responses

### Dependencies

FastAPI из коробки содержит механизм внедрения зависимостей, который позволит сократить объем кода.

**Пример 1.** Вынесем все часто используемые параметры в отдельный метод

In [ ]:
from typing import Annotated

from fastapi import Depends, FastAPI

app = FastAPI()


async def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}


@app.get("/items/")
async def read_items(commons: Annotated[dict, Depends(common_parameters)]):
    return commons


@app.get("/users/")
async def read_users(commons: Annotated[dict, Depends(common_parameters)]):
    return commons

**Пример 2.** Зависимости, сгруппированные в класс

In [ ]:
from typing import Annotated

from fastapi import Depends, FastAPI

app = FastAPI()


fake_items_db = [{"item_name": "Foo"}, {"item_name": "Bar"}, {"item_name": "Baz"}]


class CommonQueryParams:
    def __init__(self, q: str | None = None, skip: int = 0, limit: int = 100):
        self.q = q
        self.skip = skip
        self.limit = limit


@app.get("/items/")
async def read_items(commons: Annotated[CommonQueryParams, Depends(CommonQueryParams)]):
    response = {}
    if commons.q:
        response.update({"q": commons.q})
    items = fake_items_db[commons.skip : commons.skip + commons.limit]
    response.update({"items": items})
    return response

**Пример 3.** Вложенные зависимости

In [ ]:
from typing import Annotated

from fastapi import Cookie, Depends, FastAPI

app = FastAPI()


def query_extractor(q: str | None = None):
    return q


def query_or_cookie_extractor(
    q: Annotated[str, Depends(query_extractor)],
    last_query: Annotated[str | None, Cookie()] = None,
):
    if not q:
        return last_query
    return q


@app.get("/items/")
async def read_query(
    query_or_default: Annotated[str, Depends(query_or_cookie_extractor)]
):
    return {"q_or_cookie": query_or_default}

**Пример 4.** Зависимости, использумеые для завершения работы метода, в случае если запрос не прошел валидацию

In [ ]:
from typing import Annotated

from fastapi import Depends, FastAPI, Header, HTTPException

app = FastAPI()


async def verify_token(x_token: Annotated[str, Header()]):
    if x_token != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: Annotated[str, Header()]):
    if x_key != "fake-super-secret-key":
        raise HTTPException(status_code=400, detail="X-Key header invalid")
    return x_key


@app.get("/items/", dependencies=[Depends(verify_token), Depends(verify_key)])
async def read_items():
    return [{"item": "Foo"}, {"item": "Bar"}]

### Deploy

Обычно для запуска инстанса FastAPI используется ASGI server ```uvicorn```. Проблема в том, что ```uvicorn``` хоть и поддерживает асинхронные фреймворки, а также запуск wokrer'ов, но тем не менее его возможности по работе с worker'ами оставляют желать лучшего. Для решения этой проблемы используется связка ```gunicorn``` + ```uvicorn```.

Gunicorn это WSGI-сервер, однако он умеет работать в режиме мастер-процесса, запуская и отслеживая состояние нескольких worker'ов.

![gunicorn master process](https://nicewook.github.io/post_web/Gunicorn%20Worker%20Types.files/image004.gif)

```gunicorn``` поддерживает запуск ```uvicorn``` worker`ов.

```gunicorn```:
- Запустит требуемое количество процессов
- Отследит состояние worker`ов
- Остановит работу worker`ов, которые перестали отвечать
- Поднимет новые wokrer`ы

Так как для запуска worker`ов используется ```fork()```, сокет, прослушивание которого начнет ```gunicorn```, будет доступен для всех дочерних процессов-воркеров.

***Ограничение***: gunicorn работает только под Linux (под Windows нет системного вызова ```fork()```)

In [ ]:
pip install gunicorn

In [ ]:
gunicorn app.main:app --workers 4 --worker-class uvicorn.workers.UvicornWorker --bind 0.0.0.0:80 --log-level 'error'

Пример Dockerfile

In [ ]:
FROM python:3.9
WORKDIR /code
COPY ./requirements.txt /code/requirements.txt
RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt
COPY ./app /code/app
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80"]

## Работа с БД

**Цели занятия**:
- Подключить SQLAlchemy к приложению
- Добавить запросы с простыми CRUD-операциями
- Работать с БД асинхронно
- Добавить дополнительные механизмы аутентификации

### SQLAlchemy

### Authentication schemas